## 1. DATA IMPORT
Hate speech dataset from a white supremacist forum:  https://github.com/Vicomtech/hate-speech-dataset

In [1]:
import pandas as pd
import numpy as np
import glob
import os
from tqdm.notebook import tqdm
import ntpath
import matplotlib.pyplot as plt
import nltk
import nltk.sentiment.vader as vd
import string
import re
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import download
from langdetect import detect
import spacy
from wordcloud import WordCloud
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.feature_selection import SelectKBest, chi2, RFE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    precision_score,
    recall_score,
    classification_report,
    confusion_matrix,
    plot_confusion_matrix,
    plot_precision_recall_curve,
    plot_roc_curve,
)
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import sklearn as sk
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from nltk.sentiment.vader import SentimentIntensityAnalyzer as VS
import imblearn
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
import preprocessor as pproc
from cleantext import clean
from wordcloud import WordCloud

In [2]:
tqdm.pandas()

C:\Users\marti\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
nlp = spacy.load("en_core_web_sm")

In [5]:
all_files_paths = glob.glob("hate-speech-dataset/all_files/*.txt")

In [6]:
all_files_paths = [f for f in all_files_paths if os.path.isfile(f)]

In [7]:
all_files_names = [str(ntpath.basename(f)).replace(".txt", "") for f in all_files_paths]

## 1.1 Error removal

Some sentences cannot be correctly read in Windows due to error in encoding. Since they are few, we will just remove them.

In [8]:
txt_content = {}
errors = []
for name, path in tqdm(list(zip(all_files_names, all_files_paths))):
    with open(path, "r") as txt:
        try:
            txt_content[name] = txt.readline().replace(",", "")
        except Exception as ex:
            errors.append((name, str(ex)))

In [9]:
errors_list = [err[0] for err in errors]

In [10]:
df = pd.DataFrame.from_dict(txt_content, orient='index').reset_index()
df.columns = ["file_id","text"]

In [11]:
ann = pd.read_csv('hate-speech-dataset/annotations_metadata.csv')

In [12]:
ann = ann[~ann['file_id'].isin(errors_list)]

In [13]:
data = pd.merge(left=ann, right=df, left_on='file_id', right_on='file_id')

In [14]:
data['label'].unique()

array(['noHate', 'hate', 'idk/skip', 'relation'], dtype=object)

In [15]:
data = data.loc[(data["label"] != "relation") & (data["label"] != "idk/skip")]

In [16]:
data.drop(columns=["file_id", "user_id", "subforum_id", "num_contexts"], inplace=True)

In [17]:
data['label'] = data.apply(lambda x: 0 if x['label'] == "noHate" else 1, axis=1)

In [18]:
data.head()

,label,text
0,0,As of March 13th 2014 the booklet had been d...
1,0,In order to help increase the booklets downloa...
2,0,( Simply copy and paste the following text int...
3,1,Click below for a FREE download of a colorfull...
4,0,Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...


## 2. CLEANING

### 2.1 Removing links, tags, numbers and bias

In [19]:
def expand_contractions(text):
    cList = {
        "n't": " not",
        "/TD": " ",
        " PM ": " personal message ",
        " pm ": " personal message ",
        "PM ": "personal message ",
        " Donot ": " do not ",
        " MB ": " megabytes ",
        "I'm" : "I am"
    }
    
    c_re = re.compile("(%s)" % "|".join(cList.keys()))

    return c_re.sub(lambda match: cList[match.group(0)], text)

def full_text_clean(text):
    aa = expand_contractions(text)
    
    bb = pproc.clean(
        clean(pproc.clean(aa),
              fix_unicode=True,               # fix various unicode errors
              to_ascii=True,                  # transliterate to closest ASCII representation
              lower=True,                     # lowercase text
              no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
              no_urls=True,                  # replace all URLs with a special token
              no_emails=True,                # replace all email addresses with a special token
              no_phone_numbers=False,         # replace all phone numbers with a special token
              no_numbers=False,               # replace all numbers with a special token
              no_digits=False,                # replace all digits with a special token
              no_currency_symbols=False,      # replace all currency symbols with a special token
              no_punct=True,                 # remove punctuations
              replace_with_url=" ",
              replace_with_email=" ",
        )
    )
    
    #swords = stopwords.words("english")
    #swords.extend(string.punctuation)
    swords = string.punctuation

    cc = (
        bb.lower()
        .replace(r"(@[a-z0-9]+)\w+", " ")
        .replace(r"http\S+", " ")
        .replace(r"www\S+", " ")
        .replace(r"com/watch", " ")
        .replace(r"\S*[.,:;!?-]\S*[^\s\.,:;!?-]", " ")
        .replace(r" th ", " ")
        .replace(r"\w*\d\w*", " ")
        .replace(r"rlm", " ")
        .replace(r"pttm", " ")
        .replace(r"ghlight", " ")
        .replace(r"[0-9]+(?:st| st|nd| nd|rd| rd|th| th)", " ")
        .replace(r"([^a-z \t])", " ")
        .replace(r" +", " ")
        )
    
    cc = " ".join([i for i in cc.split() if not i in swords and len(i) >1 ]) # 
    
    return cc

In [20]:
data['text_clean'] = data["text"].progress_apply(lambda x: full_text_clean(x))

In [21]:
data.head()

,label,text,text_clean
0,0,As of March 13th 2014 the booklet had been d...,as of march the booklet had been downloaded ov...
1,0,In order to help increase the booklets downloa...,in order to help increase the booklets downloa...
2,0,( Simply copy and paste the following text int...,simply copy and paste the following text into ...
3,1,Click below for a FREE download of a colorfull...,click below for free download of colorfully il...
4,0,Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...,click on the download megabytes green banner link


In [22]:
data['word_count_before'] = data['text'].apply(lambda x: len(x.split()))

In [23]:
data['word_count'] = data['text_clean'].apply(lambda x: len(x.split()))

In [24]:
data['word_cleaning'] = data['word_count_before'] - data['word_count']

In [25]:
data[['word_count_before','word_count','word_cleaning']].describe()

,word_count_before,word_count,word_cleaning
count,10694.000000,10694.000000,10694.000000
mean,17.636993,14.976155,2.660838
std,13.349767,11.502731,3.513010
min,1.000000,0.000000,-1.000000
25%,9.000000,7.000000,1.000000
50%,15.000000,13.000000,2.000000
75%,24.000000,21.000000,3.000000
max,343.000000,286.000000,108.000000


In [26]:
data = data.loc[data['word_count'] > 0, ]

### 2.2 Lemmatizer and Tokenization

In [27]:
pipe = nlp.pipe(data['text_clean'], n_process=2, batch_size=2000)

In [28]:
docs = [x for x in tqdm(pipe)]

In [29]:
data['spacy_doc'] = docs

In [30]:
data['POS_spacy'] = data['spacy_doc'].progress_apply(lambda x: [(y.text, y.pos_) for y in x])

In [31]:
data['lemmatized'] = data['spacy_doc'].progress_apply(lambda x: " ".join([y.lemma_ for y in x if len(x)>1]))

In [32]:
data['tokens'] = data['spacy_doc'].progress_apply(lambda x: [y.text for y in x])

In [33]:
data['language'] = data['spacy_doc'].progress_apply(lambda x: set([y.lang_ for y in x]))

In [34]:
length = {}
for line in data['language']:
    if len(line) in length:
        length[len(line)] += 1
    else:
        length[len(line)] = 1
length

{1: 10573}

In [35]:
data['language'] = data['language'].progress_apply(lambda x: list(x)[0])

In [36]:
data['language'].unique()

array(['en'], dtype=object)

## 3. POS

In [37]:
def filter_text_pos(x):
    final_pos_text = []
    for elem in x:
        for pos in pos_list:
            if elem[1] == pos:
                final_pos_text.append(elem[0])
    
    return " ".join(final_pos_text)

In [38]:
pos_list = ["NOUN"]
data["NOUN"] = data.apply(lambda x: filter_text_pos(x["POS_spacy"]), axis=1)
data['NOUN_count'] = data['NOUN'].apply(lambda x: len(x.split()))

In [39]:
pos_list = ["PROPN"]
data["PROPN"] = data.apply(lambda x: filter_text_pos(x["POS_spacy"]), axis=1)
data['PROPN_count'] = data['PROPN'].apply(lambda x: len(x.split()))

In [40]:
pos_list = ["VERB"]
data["VERB"] = data.apply(lambda x: filter_text_pos(x["POS_spacy"]), axis=1)
data['VERB_count'] = data['VERB'].apply(lambda x: len(x.split()))

In [41]:
pos_list = ["ADJ"]
data["ADJ"] = data.apply(lambda x: filter_text_pos(x["POS_spacy"]), axis=1)
data['ADJ_count'] = data['ADJ'].apply(lambda x: len(x.split()))

In [42]:
pos_list = ["ADV"]
data["ADV"] = data.apply(lambda x: filter_text_pos(x["POS_spacy"]), axis=1)
data['ADV_count'] = data['ADV'].apply(lambda x: len(x.split()))

In [43]:
pos_list = ["PRON"]
data["PRON"] = data.apply(lambda x: filter_text_pos(x["POS_spacy"]), axis=1)
data['PRON_count'] = data['PRON'].apply(lambda x: len(x.split()))

In [44]:
pos_list = ["SCONJ"]
data["SCONJ"] = data.apply(lambda x: filter_text_pos(x["POS_spacy"]), axis=1)
data['SCONJ_count'] = data['SCONJ'].apply(lambda x: len(x.split()))

In [45]:
pos_list = ["INTJ"]
data["INTJ"] = data.apply(lambda x: filter_text_pos(x["POS_spacy"]), axis=1)
data['INTJ_count'] = data['SCONJ'].apply(lambda x: len(x.split()))

In [46]:
data.head()

,label,text,text_clean,word_count_before,word_count,word_cleaning,spacy_doc,POS_spacy,lemmatized,tokens,...,ADJ,ADJ_count,ADV,ADV_count,PRON,PRON_count,SCONJ,SCONJ_count,INTJ,INTJ_count
0,0,As of March 13th 2014 the booklet had been d...,as of march the booklet had been downloaded ov...,16,12,4,"(as, of, march, the, booklet, had, been, downl...","[(as, SCONJ), (of, ADP), (march, PROPN), (the,...",as of march the booklet have be download over ...,"[as, of, march, the, booklet, had, been, downl...",...,,0,,0,,0,as,1,,1
1,0,In order to help increase the booklets downloa...,in order to help increase the booklets downloa...,34,33,1,"(in, order, to, help, increase, the, booklets,...","[(in, ADP), (order, NOUN), (to, PART), (help, ...",in order to help increase the booklet download...,"[in, order, to, help, increase, the, booklets,...",...,great uploaded,2,,0,it who,2,if,1,,1
2,0,( Simply copy and paste the following text int...,simply copy and paste the following text into ...,15,13,2,"(simply, copy, and, paste, the, following, tex...","[(simply, ADV), (copy, VERB), (and, CCONJ), (p...",simply copy and paste the follow text into -PR...,"[simply, copy, and, paste, the, following, tex...",...,youtube,1,simply,1,,0,,0,,0
3,1,Click below for a FREE download of a colorfull...,click below for free download of colorfully il...,22,18,4,"(click, below, for, free, download, of, colorf...","[(click, VERB), (below, ADV), (for, ADP), (fre...",click below for free download of colorfully il...,"[click, below, for, free, download, of, colorf...",...,free zionistengineered intentional western,4,below colorfully,2,,0,,0,,0
4,0,Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...,click on the download megabytes green banner link,14,8,6,"(click, on, the, download, megabytes, green, b...","[(click, VERB), (on, ADP), (the, DET), (downlo...",click on the download megabyte green banner link,"[click, on, the, download, megabytes, green, b...",...,green,1,,0,,0,,0,,0


In [47]:
data.drop(columns=["spacy_doc"], inplace=True)

In [48]:
data = data[["label", "text", "text_clean", "POS_spacy", "lemmatized", "tokens", "language", "word_count_before","word_count", "word_cleaning","NOUN", "NOUN_count", "PROPN", "PROPN_count", "VERB", "VERB_count", "ADJ", "ADJ_count", "ADV", "ADV_count", "PRON", "PRON_count", "SCONJ", "SCONJ_count", "INTJ", "INTJ_count"]]

In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10573 entries, 0 to 10925
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   label              10573 non-null  int64 
 1   text               10573 non-null  object
 2   text_clean         10573 non-null  object
 3   POS_spacy          10573 non-null  object
 4   lemmatized         10573 non-null  object
 5   tokens             10573 non-null  object
 6   language           10573 non-null  object
 7   word_count_before  10573 non-null  int64 
 8   word_count         10573 non-null  int64 
 9   word_cleaning      10573 non-null  int64 
 10  NOUN               10573 non-null  object
 11  NOUN_count         10573 non-null  int64 
 12  PROPN              10573 non-null  object
 13  PROPN_count        10573 non-null  int64 
 14  VERB               10573 non-null  object
 15  VERB_count         10573 non-null  int64 
 16  ADJ                10573 non-null  objec

In [50]:
with open("serialized/data.pkl", "wb") as f:
    pickle.dump(data, f)

In [51]:
data[data['text_clean'].str.contains("ca")]

,label,text,text_clean,POS_spacy,lemmatized,tokens,language,word_count_before,word_count,word_cleaning,...,ADJ,ADJ_count,ADV,ADV_count,PRON,PRON_count,SCONJ,SCONJ_count,INTJ,INTJ_count
10,1,She may or may not be a Jew but she 's certain...,she may or may not be jew but she certainly st...,"[(she, PRON), (may, VERB), (or, CCONJ), (may, ...",-PRON- may or may not be jew but -PRON- certai...,"[she, may, or, may, not, be, jew, but, she, ce...",en,38,34,4,...,other dirty,2,certainly,1,she she she her they what,6,,0,,0
13,0,I just came across this piece of garbage .,just came across this piece of garbage,"[(just, ADV), (came, VERB), (across, ADP), (th...",just come across this piece of garbage,"[just, came, across, this, piece, of, garbage]",en,9,7,2,...,,0,just,1,,0,,0,,0
18,0,I believe this can be higher since I 've read ...,believe this can be higher since ve read repor...,"[(believe, VERB), (this, DET), (can, VERB), (b...",believe this can be high since ve read report ...,"[believe, this, can, be, higher, since, ve, re...",en,26,23,3,...,higher ve free latest,4,especially,1,,0,since that,2,,2
25,0,No wonder young people ca n't get jobs .,no wonder young people ca not get jobs,"[(no, DET), (wonder, NOUN), (young, ADJ), (peo...",no wonder young people ca not get job,"[no, wonder, young, people, ca, not, get, jobs]",en,9,8,1,...,young,1,,0,,0,,0,,0
28,0,National Geographic magazines in the 1950s and...,national geographic magazines in the and early...,"[(national, PROPN), (geographic, ADJ), (magazi...",national geographic magazine in the and early ...,"[national, geographic, magazines, in, the, and...",en,18,15,3,...,geographic replete african,3,early,1,,0,,0,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10917,0,This is what Pierre Trudeau gave Canadians once .,this is what pierre trudeau gave canadians once,"[(this, DET), (is, AUX), (what, PRON), (pierre...",this be what pierre trudeau give canadian once,"[this, is, what, pierre, trudeau, gave, canadi...",en,9,8,1,...,,0,once,1,what,1,,0,,0
10918,0,I ca n't find an actual picture of it anywhere...,ca not find an actual picture of it anywhere b...,"[(ca, VERB), (not, PART), (find, VERB), (an, D...",ca not find an actual picture of -PRON- anywhe...,"[ca, not, find, an, actual, picture, of, it, a...",en,19,16,3,...,actual,1,anywhere,1,it there it,3,,0,,0
10919,0,Trudeau Saluteunknown ( -0.191 ) Another way t...,trudeau saluteunknown another way to say givin...,"[(trudeau, PROPN), (saluteunknown, PROPN), (an...",trudeau saluteunknown another way to say give ...,"[trudeau, saluteunknown, another, way, to, say...",en,27,19,8,...,,0,primarily,1,,0,,0,,0
10922,0,Wish we at least had a Marine Le Pen to vote f...,wish we at least had marine le pen to vote for...,"[(wish, VERB), (we, PRON), (at, ADP), (least, ...",wish -PRON- at least have marine le pen to vot...,"[wish, we, at, least, had, marine, le, pen, to...",en,15,13,2,...,least marine,2,,0,we,1,,0,,0
